# ML Tests

This notebook contains a small experiment with ML I wanted to run using some classifier.

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black

import pathlib
import multiprocessing

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from ecodam_py.bedgraph import BedGraphAccessor, put_dfs_on_even_grounds
from ecodam_py.ml_pipeline import ml_pipeline
from ecodam_py.eco_atac_normalization import *

<IPython.core.display.Javascript object>

## Preprocessing

In [2]:
chromatin = read_bedgraph(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Michael/Chromatin_rep1/filtered_xmap/pipeline_bedgraphs/NoBlackMask/Chromatin_WholeGenome.bedgraph"
)
theo = read_bedgraph(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Theoretical_EcoDam/New folder/Hg38.EcoDam.insilico.canonical.keynames.bg_smoothed_7kb_resampled_with_700_overlapping_bp.bedgraph"
)
theo = theo.replace("chr23", "chrX")
naked = read_bedgraph(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/Naked_WholeGenome.bedgraph"
)
nfr = read_bedgraph(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/ENCFF240YRV.sorted.bedgraph"
)

<IPython.core.display.Javascript object>

Equalize loci of the three datasets, per chromosome:

In [ ]:
chromatingb = chromatin.groupby("chr", as_index=False)
nakedgb = naked.groupby("chr", as_index=False)
theogb = theo.groupby("chr", as_index=False)
grouped = (
    (chrom, nakedgb.get_group(chr_), theogb.get_group(chr_), nfr.query("chr == @chr_"))
    for chr_, chrom in chromatingb
)
with multiprocessing.Pool(6) as pool:
    result = pool.starmap(ml_pipeline, grouped)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/hagaih/miniconda3/envs/ecodam/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/hagaih/miniconda3/envs/ecodam/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hagaih/miniconda3/envs/ecodam/lib/python3.8/multiprocessing/pool.py", line 513, in _handle_workers
    cls._maintain_pool(ctx, Process, processes, pool, inqueue,
  File "/home/hagaih/miniconda3/envs/ecodam/lib/python3.8/multiprocessing/pool.py", line 337, in _maintain_pool
    Pool._repopulate_pool_static(ctx, Process, processes, pool,
  File "/home/hagaih/miniconda3/envs/ecodam/lib/python3.8/multiprocessing/pool.py", line 326, in _repopulate_pool_static
    w.start()
  File "/home/hagaih/miniconda3/envs/ecodam/lib/python3.8/multiprocessing/process.py", line 121, in start
    self._popen = self._Popen(self)
  File "/home/hagaih/miniconda3/envs/ecodam/lib/pyth

In [ ]:
chrom_from_disk = pd.read_parquet('/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/ml_pipeline/chrom_chr1_after_same_bounds.pq')
naked_from_disk = pd.read_parquet('/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/ml_pipeline/naked_chr1_after_same_bounds.pq')
theo_from_disk = pd.read_parquet('/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/ml_pipeline/theo_chr1_after_same_bounds.pq')

In [ ]:
chrom_from_disk

In [ ]:
naked_from_disk

In [ ]:
theo_from_disk

In [ ]:
chrom_normed, naked_normed, theo_normed = normalize_with_site_density(chrom_from_disk, naked_from_disk, theo_from_disk)